In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchaudio import datasets, transforms
##import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import soundata



In [ ]:
#this code should process the data from UrbanSound8k

#the sound data is split into 10 folds,, 
#the data needs a 10-fold cross validation ,, train on folds 1-9 and then test on 10
#then train on every fold except 9 and then test on 9 
#etc etc



train = "train"
val = "val"
test = "test"

#for this dataset you should not shuffle


transform = transforms.Compose(

)



In [ ]:
#this code should prep data from NoisyDataset

In [ ]:
#Building a basic denoising model

In [ ]:
#loss function and optimizer

#use 

In [ ]:
#compare with other data
#first experiment

#how do i do this??? compare the stats with other experiments and then make
#a graph plotting all the information,

In [ ]:
#second experiment

In [ ]:
#third experiment